In [2]:
import pandas as pd
test=pd.read_csv('/kaggle/input/multi-digit-mnist/test.csv')
train=pd.read_csv('/kaggle/input/multi-digit-mnist/train.csv')
train


,Image,first_num,second_num
0,0_00.png,0,0
1,100_00.png,0,0
2,101_00.png,0,0
3,102_00.png,0,0
4,103_00.png,0,0
...,...,...,...
63995,997_98.png,9,8
63996,998_98.png,9,8
63997,999_98.png,9,8
63998,99_98.png,9,8


In [3]:
train_images="/kaggle/input/multi-digit-mnist/train_images/"+train.iloc[:,0].values

In [4]:
train_images[0]=='/kaggle/input/multi-digit-mnist/train_images/0_00.png'

True

In [5]:
test_images="/kaggle/input/multi-digit-mnist/test_images/"+test.iloc[:,0].values

In [6]:
test_images[0]=='/kaggle/input/multi-digit-mnist/test_images/0_02.png'

True

In [7]:
train_labels = train.iloc[:, 1:].values
test_labels = test.iloc[:, 1:].values

In [8]:
train

,Image,first_num,second_num
0,0_00.png,0,0
1,100_00.png,0,0
2,101_00.png,0,0
3,102_00.png,0,0
4,103_00.png,0,0
...,...,...,...
63995,997_98.png,9,8
63996,998_98.png,9,8
63997,999_98.png,9,8
63998,99_98.png,9,8


In [9]:
train_labels[:10]

array([[0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0]])

In [10]:
test_labels[:10]

array([[0, 2],
       [0, 2],
       [0, 2],
       [0, 2],
       [0, 2],
       [0, 2],
       [0, 2],
       [0, 2],
       [0, 2],
       [0, 2]])

In [11]:
 
def read_image(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_image(image, channels=1, dtype=tf.float32)
 
    image.set_shape((64, 64, 1))
    label[0].set_shape([])
    label[1].set_shape([])

    labels = {"first_num": label[0], "second_num": label[1]}
    return image, labels

In [12]:
BATCH_SIZE=64

In [13]:
import tensorflow as tf
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = (
    train_dataset.shuffle(buffer_size=len(train_labels))
    .map(read_image)
    .batch(batch_size=BATCH_SIZE)
    .prefetch(buffer_size=AUTOTUNE)
)

test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_dataset = (
    test_dataset.map(read_image)
    .batch(batch_size=BATCH_SIZE)
    .prefetch(buffer_size=AUTOTUNE)
)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [14]:
from tensorflow import keras

In [15]:
inputs=keras.layers.Input(shape=(64,64,1))
x=keras.layers.Conv2D(filters=32,kernel_size=3,padding='same')(inputs)
x=keras.layers.BatchNormalization()(x)
# x=keras.activation.LeakyRelu()(x)
x=keras.layers.Conv2D(filters=64,kernel_size=3,padding='same')(x)
x=keras.layers.BatchNormalization()(x)
# x=keras.activation.LeakyRelu()(x)
x=keras.layers.MaxPooling2D()(x)
x=keras.layers.Conv2D(filters=64,kernel_size=3,padding='same')(x)
x=keras.layers.MaxPooling2D()(x)
x=keras.layers.Flatten()(x)
x=keras.layers.Dense(512,activation='relu')(x)
x=keras.layers.Dense(0.3)(x)
x=keras.layers.Dense(128,activation='relu')(x)
output1=keras.layers.Dense(10,name='first_num')(x)
output2=keras.layers.Dense(10,name='second_num')(x)




In [16]:
model=keras.Model(inputs=inputs,outputs=[output1,output2])

In [17]:
model.compile(loss=[keras.losses.SparseCategoricalCrossentropy(),keras.losses.SparseCategoricalCrossentropy()],optimizer=keras.optimizers.Adam(lr=0.001),metrics=['accuracy'])

In [18]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64, 64, 1)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 64, 64, 32)   320         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 64, 64, 32)  128         ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 conv2d_1 (Conv2D)              (None, 64, 64, 64)   18496       ['batch_normalization[0][0]']

In [22]:
model.fit(train_dataset,epochs=3,batch_size=64)

Epoch 1/3
1000/1000 [==============================] - 23s 23ms/step - loss: 4.6052 - first_num_loss: 2.3026 - second_num_loss: 2.3026 - first_num_accuracy: 0.1094 - second_num_accuracy: 0.1094
Epoch 2/3
1000/1000 [==============================] - 23s 23ms/step - loss: 4.6052 - first_num_loss: 2.3026 - second_num_loss: 2.3026 - first_num_accuracy: 0.1094 - second_num_accuracy: 0.1094
Epoch 3/3
1000/1000 [==============================] - 23s 23ms/step - loss: 4.6052 - first_num_loss: 2.3026 - second_num_loss: 2.3026 - first_num_accuracy: 0.1094 - second_num_accuracy: 0.1094


In [23]:
model.evaluate(test_dataset)

313/313 [==============================] - 4s 13ms/step - loss: 4.6052 - first_num_loss: 2.3026 - second_num_loss: 2.3026 - first_num_accuracy: 0.0500 - second_num_accuracy: 0.0500


[4.60518217086792,
 2.30259108543396,
 2.30259108543396,
 0.05000000074505806,
 0.05000000074505806]